<a href="https://colab.research.google.com/github/LinusBach/SentimentAnalysis/blob/main/sentiAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple sentiment analysis

Sentiment analysis, using iMDB database

First, implement and train a feedforward NN model with TF-IDF. And then train your
model using word2vec embedding. Report both training and development accuracy on
the dataset. Try to use stochastic gradient descent or (mini-batch) stochastic gradient
descent!

In [11]:
!pip install tensorflow
!pip install nltk
!pip install gensim

[nltk_data] Downloading package stopwords to /Users/uni/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### imports


In [118]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow import keras
import nltk
nltk.download('stopwords')
import csv
import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer
import gensim

[nltk_data] Downloading package stopwords to /Users/uni/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### load dataset into memory
return a list of docs and a list of respective labels

In [3]:
def load_data (filename):
  content = list()
  labels = list()

  has_header = True
  # detect if file has a header
  # with open(filename, 'r') as file:
  #   sample = file.read(64)
  #   has_header = csv.Sniffer().has_header(sample)

  with open(filename, 'r') as file:
    reader = csv.reader(file)
    # skip first line if file has a header 
    if has_header:
      next(reader)
    for c, l in reader:
      content.append(c)
      labels.append(l)
  return content, labels

### turn a dataset into clean tokens

In [4]:
def clean_data(data):
  corpus = list()
  corp_voc = dict()
  stop = nltk.corpus.stopwords.words("english")
  # regex tokenizer, find words, numbers and words containing '
  tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+(?:'\w)?")
  for doc in data:
    doc = tokenizer.tokenize(doc)
    doc_cleaned = dict()
    for tok in doc:
      # make all words lower case
      tok = tok.lower()
      # filter out numbers 
      if not tok.isdigit() and tok not in stop:
        # add clean token to document
        if tok in doc_cleaned:
          doc_cleaned[tok] += 1
        else:
          doc_cleaned[tok] = 1
    for tok in doc_cleaned.keys():
    # increase corpus vocabulary
        if tok in corp_voc:
          corp_voc[tok] += 1
        else:
          corp_voc[tok] = 1
    corpus.append(doc_cleaned)
  return corpus, corp_voc

# filter all words out of a corpus that are not in a vocabulary
def get_filtered_corpus(corpus, vocab):
  clean_corpus = list()
  for doc in corpus:
    clean_doc = dict()
    for tok in doc:
      if tok in vocab:
        clean_doc[tok] = doc[tok]
    clean_corpus.append(clean_doc)
  return clean_corpus

### preprocess the dataset

tf-idf vectorizer

In [5]:
# corpus must be a list of dicts of form (token: occurences)
# vocab must be a dict of form (token: documents in corpus containing token)
def preprocess_tf_idf(corpus, vocab):
  processed = np.zeros((len(corpus), len(vocab)))
  idf = get_idf(vocab)
  token_order = {tok: i for i, tok, in enumerate(sorted(vocab.keys()))}
  for n_doc, doc in enumerate(corpus):
    tf = get_tf(doc)
    for tok in set(doc):
      tok_pos = token_order[tok]
      processed[n_doc][tok_pos] = tf[tok]*idf[tok]
  return processed

def get_tf(doc):
  tf = dict()
  for tok, occ in doc.items():
    tf[tok] = occ / len(doc)
  return tf

def get_idf(corp_voc):
  idf = dict()
  for tok, docs_containing in corp_voc.items():
    idf[tok] = np.log10(len(corp_voc) / docs_containing)
  return idf


semantic embeddings
pooling word2vec embeddings

In [148]:
def preprocess_sem_vec(corpus):
  processed = list()
  model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
  total_used = 0
  total_available = 0
  for n_doc, doc in enumerate(corpus):
    vecs = [occ*model[tok] for tok, occ in doc.items() if tok in model.key_to_index]
    total_used += len(vecs)
    total_available += len(doc.items())
    processed.append(np.mean(vecs, axis=0))

  return np.array(processed)

load training data

In [137]:
# read data from file
raw_data, labels = load_data("Train.csv")
full_corpus, full_vocab = clean_data(raw_data)

# select part of vocabulary
frequencies = sorted(full_vocab.items(), key=lambda x : x[1], reverse=True)
# number of most common non-stop words that we consider
VOCAB_SIZE = 5000
# number of most frequent words to be disregarded, but this is now irrelevant due to the removal of stop words
HIGHER_CUTOFF = 0
vocab = {x[0] : x[1] for x in frequencies[HIGHER_CUTOFF:HIGHER_CUTOFF+VOCAB_SIZE]}

corpus = get_filtered_corpus(full_corpus, vocab.keys())
# process labels
taining_labels = np.array([0 if int(label) == 0 else 1 for label in labels]) # keras.utils.to_categorical(labels, num_classes=2)

load validation data

In [50]:
v_raw, v_labels = load_data("Valid.csv")
full_valid_corpus, _ = clean_data(v_raw)
valid_corpus = get_filtered_corpus(full_valid_corpus, vocab)
valid_labels = np.array([0 if int(label) == 0 else 1 for label in v_labels]) # keras.utils.to_categorical(v_labels, num_classes=2)

load testing data

In [51]:
t_raw, t_labels = load_data("Test.csv")
full_test_corpus, _ = clean_data(t_raw)
test_corpus = get_filtered_corpus(full_test_corpus, vocab)
test_labels = np.array([0 if int(label) == 0 else 1 for label in t_labels]) # keras.utils.to_categorical(t_labels, num_classes=2)

define and preprocess training, testing and validation data

In [52]:
# get tf-idf representation of data
tfidf_data = preprocess_tf_idf(corpus, vocab)
test_data = preprocess_tf_idf(test_corpus, vocab)
valid_data = preprocess_tf_idf(valid_corpus, vocab)

In [149]:
# get semantic embeddings of data
w2v_train_data = preprocess_sem_vec(corpus)
w2v_valid_data = preprocess_sem_vec(valid_corpus)
w2v_test_data = preprocess_sem_vec(test_corpus)

### classify a review as negative or positive.

In [68]:
def predict_sentiment(model, doc):
  return model.predict(doc)

### define the model


In [179]:
def define_tf_idf_model(dropout=0.2, l1=1e-5, l2=1e-4):
  model = keras.models.Sequential()
  model.add(layers.Dropout(dropout))
  model.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2)))
  model.add(layers.Dropout(dropout))
  model.add(layers.Dense(1, activation='sigmoid', kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2)))
  return model

## Running the model

Setting constants

In [184]:
TF_LEARNING_RATE = 4e-4
TF_BATCH_SIZE = 512
TF_EPOCHS = 100

TF_DROPOUT = 0.25
TF_L1 = 5e-6
TF_L2 = 5e-5

In [185]:
# build model
tfidf_model = define_tf_idf_model(dropout=TF_DROPOUT, l1=TF_L1, l2=TF_L2)
tfidf_model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=TF_LEARNING_RATE),
    metrics=['accuracy'])
# train model
print("begin training")
tfidf_model.fit(
    tfidf_data,
    taining_labels,
    validation_data=(valid_data, valid_labels),
    epochs=TF_EPOCHS,
    batch_size=TF_BATCH_SIZE)
print("begin evaluation")
tfidf_model.evaluate(test_data, test_labels)

79/79 [==============================] - 1s 10ms/step - loss: 0.6147 - accuracy: 0.8171 - val_loss: 0.6086 - val_accuracy: 0.8412
Epoch 13/100
79/79 [==============================] - 1s 10ms/step - loss: 0.6059 - accuracy: 0.8238 - val_loss: 0.6006 - val_accuracy: 0.8438
Epoch 14/100
79/79 [==============================] - 1s 10ms/step - loss: 0.5986 - accuracy: 0.8285 - val_loss: 0.5931 - val_accuracy: 0.8478
Epoch 15/100
79/79 [==============================] - 1s 10ms/step - loss: 0.5910 - accuracy: 0.8319 - val_loss: 0.5857 - val_accuracy: 0.8494
Epoch 16/100
79/79 [==============================] - 1s 10ms/step - loss: 0.5855 - accuracy: 0.8333 - val_loss: 0.5793 - val_accuracy: 0.8506
Epoch 17/100
79/79 [==============================] - 1s 10ms/step - loss: 0.5795 - accuracy: 0.8361 - val_loss: 0.5731 - val_accuracy: 0.8542
Epoch 18/100
79/79 [==============================] - 1s 10ms/step - loss: 0.5736 - accuracy: 0.8376 - val_loss: 0.5675 - val_accuracy: 0.8538
Epoch 19/100

[0.42469364404678345, 0.8913999795913696]

begin training
Epoch 1/100
79/79 [==============================] - 1s 11ms/step - loss: 0.7085 - accuracy: 0.6635 - val_loss: 0.6948 - val_accuracy: 0.7794
Epoch 2/100
79/79 [==============================] - 1s 10ms/step - loss: 0.6907 - accuracy: 0.6927 - val_loss: 0.6878 - val_accuracy: 0.7812
Epoch 3/100
79/79 [==============================] - 1s 9ms/step - loss: 0.6855 - accuracy: 0.7419 - val_loss: 0.6828 - val_accuracy: 0.7846
Epoch 4/100
79/79 [==============================] - 1s 9ms/step - loss: 0.6801 - accuracy: 0.7570 - val_loss: 0.6767 - val_accuracy: 0.7612
Epoch 5/100
79/79 [==============================] - 1s 10ms/step - loss: 0.6737 - accuracy: 0.7636 - val_loss: 0.6693 - val_accuracy: 0.7914
Epoch 6/100
79/79 [==============================] - 1s 10ms/step - loss: 0.6655 - accuracy: 0.7753 - val_loss: 0.6609 - val_accuracy: 0.8048
Epoch 7/100
79/79 [==============================] - 1s 10ms/step - loss: 0.6570 - accuracy: 0.7829 - val_loss: 0.6518 - val_accuracy: 

[0.4239771068096161, 0.8902000188827515]

train a model on word2vec embeddings

In [186]:
def define_word2vec_model(dropout=0.4, l1=1e-5, l2=1e-4):
  model = keras.models.Sequential()
  model.add(layers.Dropout(dropout))
  model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2)))
  model.add(layers.Dropout(dropout))
  model.add(layers.Dense(128, activation='relu', kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2)))
  model.add(layers.Dropout(dropout))
  model.add(layers.Dense(1, activation='sigmoid', kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2)))
  return model

In [189]:
W2V_LEARNING_RATE = 2e-4
W2V_BATCH_SIZE = 512
W2V_EPOCHS = 100

W2V_DROPOUT = 0.1
W2V_L1 = 2e-5
W2V_L2 = 1e-4

In [190]:
# build model
w2v_model = define_word2vec_model(dropout=W2V_DROPOUT, l1=W2V_L1, l2=W2V_L2)
w2v_model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=W2V_LEARNING_RATE),
    metrics=['accuracy'])
# train model
print("begin training")
w2v_model.fit(
    w2v_train_data,
    taining_labels,
    validation_data=(w2v_valid_data, valid_labels),
    epochs=W2V_EPOCHS,
    batch_size=W2V_BATCH_SIZE)
print("begin evaluation")
w2v_model.evaluate(w2v_test_data, test_labels)

begin training
Epoch 1/100
79/79 [==============================] - 1s 5ms/step - loss: 0.8013 - accuracy: 0.7027 - val_loss: 0.6961 - val_accuracy: 0.7980
Epoch 2/100
79/79 [==============================] - 0s 4ms/step - loss: 0.6277 - accuracy: 0.7897 - val_loss: 0.5550 - val_accuracy: 0.8172
Epoch 3/100
79/79 [==============================] - 0s 4ms/step - loss: 0.5546 - accuracy: 0.8091 - val_loss: 0.5098 - val_accuracy: 0.8322
Epoch 4/100
79/79 [==============================] - 0s 4ms/step - loss: 0.5273 - accuracy: 0.8188 - val_loss: 0.4874 - val_accuracy: 0.8432
Epoch 5/100
79/79 [==============================] - 0s 4ms/step - loss: 0.5114 - accuracy: 0.8240 - val_loss: 0.4739 - val_accuracy: 0.8462
Epoch 6/100
79/79 [==============================] - 0s 4ms/step - loss: 0.4995 - accuracy: 0.8266 - val_loss: 0.4643 - val_accuracy: 0.8472
Epoch 7/100
79/79 [==============================] - 0s 5ms/step - loss: 0.4924 - accuracy: 0.8269 - val_loss: 0.4571 - val_accuracy: 0.849

[0.3807304799556732, 0.8555999994277954]